In [27]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [28]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [29]:
tf.keras.backend.clear_session()

In [30]:
subject = 'Bahia - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [31]:
data = pd.read_csv('2003_mo_model_input_BA.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Bahia - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Bahia - IDH,Bahia - Produção de Cimento (t),Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,Bahia - Desemprego,Bahia - Consumo de Cimento (t)
0,2003-1,0.299858,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,0.669899,39.798880,1.317344e+08,8.384593e+06,8.566149,1.216359e+08,8.348779,151.297
1,2003-2,0.301903,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,0.670210,39.480034,1.318964e+08,8.391946e+06,8.569210,1.216914e+08,8.342979,138.707
2,2003-3,0.303709,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,0.670521,39.400256,1.320584e+08,8.399299e+06,8.572270,1.217469e+08,8.337179,135.009
3,2003-4,0.305311,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,0.670831,39.417185,1.322204e+08,8.406652e+06,8.575331,1.218023e+08,8.331379,126.554
4,2003-5,0.306860,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,0.671142,39.479943,1.323824e+08,8.414005e+06,8.578392,1.218578e+08,8.325579,137.331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,0.597113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106.069163,NaN,NaN,NaN,NaN,NaN,366.305
236,2022-9,0.596178,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.752943,NaN,NaN,NaN,NaN,NaN,346.042
237,2022-10,0.594662,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.537361,NaN,NaN,NaN,NaN,NaN,347.901
238,2022-11,0.592436,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104.971241,NaN,NaN,NaN,NaN,NaN,310.845


In [32]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Bahia - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Bahia - IDH,Bahia - Produção de Cimento (t),Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,Bahia - Desemprego
0,-1.440329,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-2.143224,-1.723080,-1.703176,-0.816345,-2.235724,-2.158201,-0.884052
1,-1.404914,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-2.108465,-1.735985,-1.684201,-0.772933,-2.194204,-2.117854,-0.885724
2,-1.373637,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-2.073707,-1.739214,-1.665225,-0.729522,-2.152684,-2.077508,-0.887395
3,-1.345888,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-2.038948,-1.738529,-1.646249,-0.686111,-2.111164,-2.037161,-0.889067
4,-1.319065,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-2.004190,-1.735989,-1.627274,-0.642700,-2.069645,-1.996814,-0.890739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.796141,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.261516,0.511535,1.096947,-1.668064,0.877256,0.724962,1.200853
188,1.811284,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.253736,0.513264,1.085441,-1.663847,0.851021,0.704849,1.200487
189,1.833479,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.245956,0.526782,1.073935,-1.659631,0.824786,0.684737,1.200122
190,1.853109,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.238176,0.525379,1.062429,-1.655414,0.798552,0.664624,1.199756


In [33]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      129.857
1      126.813
2      152.225
3      136.288
4      148.117
        ...   
235        NaN
236        NaN
237        NaN
238        NaN
239        NaN
Name: Bahia - Consumo de Cimento (t), Length: 240, dtype: float64

In [34]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Bahia - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Bahia - IDH,Bahia - Produção de Cimento (t),Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,Bahia - Desemprego
0,-1.440329,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-2.143224,-1.723080,-1.703176,-0.816345,-2.235724,-2.158201,-0.884052
1,-1.404914,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-2.108465,-1.735985,-1.684201,-0.772933,-2.194204,-2.117854,-0.885724
2,-1.373637,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-2.073707,-1.739214,-1.665225,-0.729522,-2.152684,-2.077508,-0.887395
3,-1.345888,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-2.038948,-1.738529,-1.646249,-0.686111,-2.111164,-2.037161,-0.889067
4,-1.319065,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-2.004190,-1.735989,-1.627274,-0.642700,-2.069645,-1.996814,-0.890739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,1.097864,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,1.430739,0.617097,1.205171,-1.332406,1.004714,1.051323,1.273469
158,1.120412,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,1.428086,0.590103,1.206522,-1.357999,1.007770,1.045502,1.268173
159,1.139591,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,1.425433,0.561454,1.207873,-1.383592,1.010826,1.039682,1.262877
160,1.156903,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,1.422780,0.523492,1.209225,-1.409185,1.013881,1.033862,1.257581


In [35]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      129.857
1      126.813
2      152.225
3      136.288
4      148.117
        ...   
157    213.266
158    285.938
159    219.576
160    267.203
161    240.714
Name: Bahia - Consumo de Cimento (t), Length: 162, dtype: float64

In [36]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [37]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 15)

In [38]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,Bahia - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Bahia - IDH,Bahia - Produção de Cimento (t),Bahia - PIB - Estadual,Bahia - PIB - Construção Civil,Bahia - PIB - Per Capita,Bahia - PIB - Preços de Mercado,Bahia - Desemprego
126,0.558660,0.888984,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243,0.461215,1.205505,0.833268,0.194482,0.840990,1.014636,0.957509
127,0.586295,0.954254,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828,0.471044,1.203037,0.852943,0.147537,0.856945,1.027926,0.975019
128,0.614441,1.045217,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225,0.480874,1.178322,0.872618,0.100591,0.872900,1.041216,0.992528
129,0.639726,1.176395,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522,0.490704,1.155459,0.892293,0.053646,0.888855,1.054506,1.010038
130,0.666428,1.303259,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757,0.500533,1.124580,0.911968,0.006700,0.904809,1.067796,1.027547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.796141,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.261516,0.511535,1.096947,-1.668064,0.877256,0.724962,1.200853
188,1.811284,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.253736,0.513264,1.085441,-1.663847,0.851021,0.704849,1.200487
189,1.833479,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.245956,0.526782,1.073935,-1.659631,0.824786,0.684737,1.200122
190,1.853109,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.238176,0.525379,1.062429,-1.655414,0.798552,0.664624,1.199756


In [39]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 15)

In [40]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [41]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(36, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(180,activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [42]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(50)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [43]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[953385605, 1398630587, 4207512635, 3017095094, 2962844844, 669927086, 933767292, 3850737802, 2736327757, 1432600812, 2669581153, 2468202985, 1950448457, 3444325007, 1313389649, 2758518568, 1645085125, 913255524, 1612852545, 1441819112, 2820858924, 3040910246, 1554048202, 3647463482, 327623753, 1602427950, 1337446084, 912079304, 710858519, 993367160, 104275640, 168432183, 2710218074, 770163702, 2246952251, 2966525029, 2562594440, 2433259551, 716854720, 4007910087, 4287914308, 1693856184, 3605775565, 2337026775, 2454287470, 3602619436, 4165480400, 432513492, 4259601822, 3289162394]


Step: 0 ___________________________________________
val_loss: 634.12646484375
winner_seed: 953385605


Step: 1 ___________________________________________
val_loss: 585.4969482421875
winner_seed: 1398630587


Step: 2 ___________________________________________
val_loss: 646.9217529296875


Step: 3 ___________________________________________
val_loss: 697.1376342773438


Step: 4 _____________________________

In [44]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 129ms/step - loss: 60717.0781 - val_loss: 2915.0876
Epoch 2/10000
4/4 [==============================] - 0s 10ms/step - loss: 3008.0212 - val_loss: 3088.8416
Epoch 3/10000
4/4 [==============================] - 0s 9ms/step - loss: 3172.4160 - val_loss: 2939.7595
Epoch 4/10000
4/4 [==============================] - 0s 9ms/step - loss: 2999.5115 - val_loss: 4149.3423
Epoch 5/10000
4/4 [==============================] - 0s 9ms/step - loss: 3493.8362 - val_loss: 2995.6143
Epoch 6/10000
4/4 [==============================] - 0s 9ms/step - loss: 2968.6030 - val_loss: 2885.0513
Epoch 7/10000
4/4 [==============================] - 0s 9ms/step - loss: 2906.4475 - val_loss: 3135.6606
Epoch 8/10000
4/4 [==============================] - 0s 10ms/step - loss: 2857.0359 - val_loss: 2733.2036
Epoch 9/10000
4/4 [==============================] - 0s 9ms/step - loss: 2627.3186 - val_loss: 2421.4419
Epoch 10/10000
4/4 [==============================

4/4 [==============================] - 0s 10ms/step - loss: 520.0431 - val_loss: 1368.8710
Epoch 79/10000
4/4 [==============================] - 0s 11ms/step - loss: 834.9229 - val_loss: 724.1659
Epoch 80/10000
4/4 [==============================] - 0s 11ms/step - loss: 452.4492 - val_loss: 865.8314
Epoch 81/10000
4/4 [==============================] - 0s 10ms/step - loss: 415.4047 - val_loss: 937.6526
Epoch 82/10000
4/4 [==============================] - 0s 11ms/step - loss: 504.7975 - val_loss: 695.2791
Epoch 83/10000
4/4 [==============================] - 0s 10ms/step - loss: 716.2485 - val_loss: 1080.8569
Epoch 84/10000
4/4 [==============================] - 0s 10ms/step - loss: 717.7508 - val_loss: 874.7429
Epoch 85/10000
4/4 [==============================] - 0s 10ms/step - loss: 685.1380 - val_loss: 717.1689
Epoch 86/10000
4/4 [==============================] - 0s 10ms/step - loss: 515.3300 - val_loss: 1237.3318
Epoch 87/10000
4/4 [==============================] - 0s 11ms/step 

4/4 [==============================] - 0s 11ms/step - loss: 497.1668 - val_loss: 1026.5957
Epoch 156/10000
4/4 [==============================] - 0s 12ms/step - loss: 421.0104 - val_loss: 744.5645
Epoch 157/10000
4/4 [==============================] - 0s 12ms/step - loss: 320.2724 - val_loss: 850.9171
Epoch 158/10000
4/4 [==============================] - 0s 11ms/step - loss: 364.5703 - val_loss: 740.5740
Epoch 159/10000
4/4 [==============================] - 0s 11ms/step - loss: 329.3864 - val_loss: 745.0852
Epoch 160/10000
4/4 [==============================] - 0s 12ms/step - loss: 307.4804 - val_loss: 744.6065
Epoch 161/10000
4/4 [==============================] - 0s 12ms/step - loss: 283.2758 - val_loss: 682.1328
Epoch 162/10000
4/4 [==============================] - 0s 11ms/step - loss: 299.6881 - val_loss: 900.8555
Epoch 163/10000
4/4 [==============================] - 0s 12ms/step - loss: 398.3363 - val_loss: 857.3642
Epoch 164/10000
4/4 [==============================] - 0s 13m

4/4 [==============================] - 0s 11ms/step - loss: 262.2484 - val_loss: 698.4492
Epoch 233/10000
4/4 [==============================] - 0s 11ms/step - loss: 310.0078 - val_loss: 676.3617
Epoch 234/10000
4/4 [==============================] - 0s 11ms/step - loss: 507.7616 - val_loss: 610.7617
Epoch 235/10000
4/4 [==============================] - 0s 11ms/step - loss: 342.0919 - val_loss: 720.3484
Epoch 236/10000
4/4 [==============================] - 0s 12ms/step - loss: 357.1886 - val_loss: 599.3671
Epoch 237/10000
4/4 [==============================] - 0s 11ms/step - loss: 358.9389 - val_loss: 716.8105
Epoch 238/10000
4/4 [==============================] - 0s 12ms/step - loss: 310.2797 - val_loss: 739.2175
Epoch 239/10000
4/4 [==============================] - 0s 12ms/step - loss: 369.1667 - val_loss: 776.3689
Epoch 240/10000
4/4 [==============================] - 0s 11ms/step - loss: 500.4246 - val_loss: 733.9523
Epoch 241/10000
4/4 [==============================] - 0s 10ms

4/4 [==============================] - 0s 11ms/step - loss: 5802.8662 - val_loss: 3522.6470
Epoch 310/10000
4/4 [==============================] - 0s 11ms/step - loss: 4112.1680 - val_loss: 3829.9558
Epoch 311/10000
4/4 [==============================] - 0s 11ms/step - loss: 4351.8481 - val_loss: 3225.5164
Epoch 312/10000
4/4 [==============================] - 0s 12ms/step - loss: 5763.3125 - val_loss: 3119.5815
Epoch 313/10000
4/4 [==============================] - 0s 12ms/step - loss: 6597.4185 - val_loss: 2911.3840
Epoch 314/10000
4/4 [==============================] - 0s 11ms/step - loss: 4596.9150 - val_loss: 2725.3271
Epoch 315/10000
4/4 [==============================] - 0s 11ms/step - loss: 4320.1826 - val_loss: 5140.6582
Epoch 316/10000
4/4 [==============================] - 0s 11ms/step - loss: 4505.7866 - val_loss: 2092.5015
Epoch 317/10000
4/4 [==============================] - 0s 10ms/step - loss: 5316.4277 - val_loss: 2152.8757
Epoch 318/10000
4/4 [=======================

...

Epoch 996/10000
4/4 [==============================] - 0s 11ms/step - loss: 1070.8810 - val_loss: 643.8918
Epoch 997/10000
4/4 [==============================] - 0s 11ms/step - loss: 1323.0350 - val_loss: 641.8818
Epoch 998/10000
4/4 [==============================] - 0s 10ms/step - loss: 820.0718 - val_loss: 754.4171
Epoch 999/10000
4/4 [==============================] - 0s 9ms/step - loss: 1063.8693 - val_loss: 879.5645
Epoch 1000/10000
4/4 [==============================] - 0s 9ms/step - loss: 1180.3120 - val_loss: 902.5629
Epoch 1001/10000
4/4 [==============================] - 0s 9ms/step - loss: 1231.8854 - val_loss: 942.5475
Epoch 1002/10000
4/4 [==============================] - 0s 9ms/step - loss: 1441.3734 - val_loss: 806.8909
Epoch 1003/10000
4/4 [==============================] - 0s 9ms/step - loss: 1024.0165 - val_loss: 798.5332
Epoch 1004/10000
4/4 [==============================] - 0s 10ms/step - loss: 1179.8378 - val_loss: 811.1418
Epoch 1005/10000
4/4 [================

4/4 [==============================] - 0s 11ms/step - loss: 1068.0579 - val_loss: 836.5331
Epoch 1073/10000
4/4 [==============================] - 0s 12ms/step - loss: 1056.8611 - val_loss: 828.0117
Epoch 1074/10000
4/4 [==============================] - 0s 11ms/step - loss: 1019.9723 - val_loss: 841.9636
Epoch 1075/10000
4/4 [==============================] - 0s 11ms/step - loss: 1005.1405 - val_loss: 804.7136
Epoch 1076/10000
4/4 [==============================] - 0s 11ms/step - loss: 1046.1069 - val_loss: 848.4452
Epoch 1077/10000
4/4 [==============================] - 0s 10ms/step - loss: 875.5311 - val_loss: 1002.2125
Epoch 1078/10000
4/4 [==============================] - 0s 10ms/step - loss: 1381.1823 - val_loss: 1124.5577
Epoch 1079/10000
4/4 [==============================] - 0s 11ms/step - loss: 1055.2292 - val_loss: 1069.8506
Epoch 1080/10000
4/4 [==============================] - 0s 12ms/step - loss: 898.1676 - val_loss: 1027.7563
Epoch 1081/10000
4/4 [=====================

4/4 [==============================] - 0s 12ms/step - loss: 1116.9020 - val_loss: 963.9536
Epoch 1149/10000
4/4 [==============================] - 0s 12ms/step - loss: 1014.4191 - val_loss: 852.8317
Epoch 1150/10000
4/4 [==============================] - 0s 11ms/step - loss: 1025.3768 - val_loss: 712.2727
Epoch 1151/10000
4/4 [==============================] - 0s 12ms/step - loss: 859.6163 - val_loss: 704.4280
Epoch 1152/10000
4/4 [==============================] - 0s 13ms/step - loss: 787.0880 - val_loss: 712.1341
Epoch 1153/10000
4/4 [==============================] - 0s 15ms/step - loss: 1176.7279 - val_loss: 652.4122
Epoch 1154/10000
4/4 [==============================] - 0s 16ms/step - loss: 1095.5597 - val_loss: 852.4103
Epoch 1155/10000
4/4 [==============================] - 0s 16ms/step - loss: 849.4373 - val_loss: 833.0650
Epoch 1156/10000
4/4 [==============================] - 0s 15ms/step - loss: 1115.4114 - val_loss: 832.0869
Epoch 1157/10000
4/4 [==========================

4/4 [==============================] - 0s 11ms/step - loss: 993.1935 - val_loss: 643.6610
Epoch 1225/10000
4/4 [==============================] - 0s 11ms/step - loss: 1002.0759 - val_loss: 602.9101
Epoch 1226/10000
4/4 [==============================] - 0s 11ms/step - loss: 1011.2747 - val_loss: 603.7141
Epoch 1227/10000
4/4 [==============================] - 0s 11ms/step - loss: 1161.3123 - val_loss: 618.7615
Epoch 1228/10000
4/4 [==============================] - 0s 11ms/step - loss: 849.4807 - val_loss: 644.8598
Epoch 1229/10000
4/4 [==============================] - 0s 10ms/step - loss: 925.6836 - val_loss: 636.2776
Epoch 1230/10000
4/4 [==============================] - 0s 11ms/step - loss: 1099.8425 - val_loss: 634.9188
Epoch 1231/10000
4/4 [==============================] - 0s 12ms/step - loss: 1063.2697 - val_loss: 641.5945
Epoch 1232/10000
4/4 [==============================] - 0s 11ms/step - loss: 1298.8502 - val_loss: 649.1331
Epoch 1233/10000
4/4 [==========================

4/4 [==============================] - 0s 11ms/step - loss: 1238.6422 - val_loss: 772.1512
Epoch 1301/10000
4/4 [==============================] - 0s 13ms/step - loss: 1336.1078 - val_loss: 771.3019
Epoch 1302/10000
4/4 [==============================] - 0s 12ms/step - loss: 1100.0038 - val_loss: 809.0584
Epoch 1303/10000
4/4 [==============================] - 0s 13ms/step - loss: 918.2456 - val_loss: 783.4503
Epoch 1304/10000
4/4 [==============================] - 0s 11ms/step - loss: 1309.2423 - val_loss: 708.0165
Epoch 1305/10000
4/4 [==============================] - 0s 11ms/step - loss: 1174.7606 - val_loss: 646.2256
Epoch 1306/10000
4/4 [==============================] - 0s 10ms/step - loss: 1070.2131 - val_loss: 699.1807
Epoch 1307/10000
4/4 [==============================] - 0s 11ms/step - loss: 1267.5187 - val_loss: 700.9276
Epoch 1308/10000
4/4 [==============================] - 0s 12ms/step - loss: 942.6499 - val_loss: 744.3565
Epoch 1309/10000
4/4 [=========================

In [45]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [46]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 15ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,264.445587,272.968475,282.495056,282.494995,282.494995,282.494995,282.494995,282.494995,283.068329,283.079132,282.024689,282.034607,283.079376,283.079346,282.036194,282.037262,282.039734,286.564301,284.209991,284.209991,278.149536,278.157318,278.157318,275.394501,278.150574,286.565521,286.565521,287.515472,287.515472,287.523132
Target,250.101,277.528,270.092,278.146,257.458,231.748,268.336,223.453,241.464,238.901,191.989,272.452,261.009,292.688,258.881,276.879,255.774,208.326,291.428,249.43,241.612,252.303,281.912,200.213,270.511,281.466,285.535,328.259,298.078,264.838
Error,14.344589,4.55954,12.403046,4.348999,25.036987,50.746994,14.158997,59.041992,41.604324,44.178131,90.03569,9.582611,22.070374,9.608643,23.155182,5.158264,26.265732,78.238297,7.218018,34.779999,36.537537,25.854324,3.754669,75.181503,7.639587,5.099518,1.030518,40.74353,10.562531,22.68512


In [47]:
display(mae)
display(mape)

26.854174

0.11451766

In [48]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [49]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

NameError: name 'test_target' is not defined